# Aula 5 - pandas

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Criar tabelas dinâmicas e fazer reshape do df (Melt, pivot, pivot_table)
- 2) Transformação de Dados (cut, qcut, get_dummies)
- 3) Utilidades Extras (multiindex to singleindex, combine_first)
_______

### Objetivos

Apresentar como criar tabelas dinâmicas, como fazer transformações em dados contínuos e categóricos e aprender como trabalhar com multiindex

____
____
____

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv("data/titanic.csv")

In [3]:
df.drop(['PassengerId','Ticket','Name'],inplace=True,axis=1)

## Construindo uma Tabela Dinâmica usando Pandas

É hora de construir uma tabela dinâmica em Python usando a incrível biblioteca Pandas! Exploraremos as diferentes facetas de uma tabela dinâmica neste artigo e construiremos uma tabela dinâmica incrível e flexível a partir do zero.


    * pivot_table requer um dado e um parâmetro de índice
    * data é o dataframe do Pandas que você passa para a função
    * índice é o recurso que permite agrupar seus dados. O recurso de índice aparecerá como um índice na tabela resultante



In [4]:
# index único
table = pd.pivot_table(data=df, index=['Sex'])
table

,Age,Fare,Parch,Pclass,SibSp,Survived
Sex,,,,,,
female,27.915709,44.479818,0.649682,2.159236,0.694268,0.742038
male,30.726645,25.523893,0.235702,2.389948,0.429809,0.188908


In [5]:
# múltiplos indexes
table = pd.pivot_table(df, index=['Sex','Pclass'])
table

Age        Fare     Parch     SibSp  Survived
Sex    Pclass                                                     
female 1       34.611765  106.125798  0.457447  0.553191  0.968085
       2       28.722973   21.970121  0.605263  0.486842  0.921053
       3       21.750000   16.118810  0.798611  0.895833  0.500000
male   1       41.281386   67.226127  0.278689  0.311475  0.368852
       2       30.740707   19.741782  0.222222  0.342593  0.157407
       3       26.507589   12.661633  0.224784  0.498559  0.135447

### Função de agregação
Por padrão o `.pivot_table()` utiliza o `np.mean()` como função de agragação, mas podemos utilizar diferentes funções de agregação para diferentes colunas. Para isso, precisamos de um dicionário como entrada para o parâmetro aggfunc com o nome da coluna como chave e a função agregada como o valor. <br>
Vamos criar uma pivot table calculando a média de 'Age' e a soma para o 'Survived':


In [6]:
# diferentes funções de agregação
table = pd.pivot_table(df, 
                       index=['Sex','Pclass'], 
                       aggfunc={'Age':np.mean, 'Survived':np.sum})
table

Age  Survived
Sex    Pclass                     
female 1       34.611765        91
       2       28.722973        70
       3       21.750000        72
male   1       41.281386        45
       2       30.740707        17
       3       26.507589        47

Qual a diferença entre esse pivot_table e um groupby?

In [7]:
df.groupby(['Sex','Pclass']).agg({'Age':"mean",'Survived':sum})

Age  Survived
Sex    Pclass                     
female 1       34.611765        91
       2       28.722973        70
       3       21.750000        72
male   1       41.281386        45
       2       30.740707        17
       3       26.507589        47

Para ficar mais parecido à tabela dinâmica do excel podemos indicar um dos index para ser visualizado como coluna e adicionar os totais de colunas e índices:

In [20]:
table = pd.pivot_table(df,
                       index=['Sex'],
                       columns=['Pclass'],
                       values=['Survived'],
                       aggfunc=np.sum,
                       margins=1)#.drop(('Survived', 'All'), axis=1)
table

Survived              
Pclass        1   2    3  All
Sex                          
female       91  70   72  233
male         45  17   47  109
All         136  87  119  342

In [32]:
(91+70+72)/233

1.0

In [18]:
table.columns

MultiIndex([('Survived',     1),
            ('Survived',     2),
            ('Survived',     3),
            ('Survived', 'All')],
           names=[None, 'Pclass'])

In [41]:
table_prob = pd.pivot_table(df,
                       index=['Sex'],
                       columns=['Pclass'],
                       values=['Survived'],
                       aggfunc=np.mean,
                       margins=1)
table_prob

Survived                              
Pclass         1         2         3       All
Sex                                           
female  0.968085  0.921053  0.500000  0.742038
male    0.368852  0.157407  0.135447  0.188908
All     0.629630  0.472826  0.242363  0.383838

In [29]:
(0.968085 + 0.921053 + 0.5)/3

0.7963793333333333

In [33]:
233/342

0.6812865497076024

Formatando nossa saída

In [28]:
(table_prob*100).style.format('{0:,.1f}%')

O `pd.pivot_table()` nos permite passar vários parâmetros úteis: <br>
pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True) <br>
Segue a mesma ideia de `pd.unstack()`.

## Desfazendo uma tabela dinâmica
Para fazer um unpivoting utilizamos o `pd.melt()`. Esse método é utilizado quando queremos que uma ou mais colunas se tornem colunas de identificadores. Segue a mesma ideia de `pd.stack()`. As colunas que vamos dissolver são definidas por `id_vars` e `value_vars`.

Parâmetros: <br>
pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)


   * __frame__: DataFrame <br>
   * __id_vars__: Colunas para serem usadas como categorias. São as colunas que você quer manter do jeito que estão. <br> 
   * __value_vars__: Colunas para o unpivot - para sairem do formato largo para longo (wide to long). Se não especificada, usa todas as colunas que não estão em id_vars. <br> 
   * __var_name__: Nome para a nova coluna de variáveis categóricas. <br>
   * __value_name__: Nome para ser utilizado na coluna de valores. <br>
   * __col_level__: Se as colunas são MultiIndex.<br>

Vamos simplificar nossa tabela de probabilidades eliminando a coluna e linha com os totais, resetando o index e eliminando o multi-index do nome das colunas renomeando-as.

In [42]:
table_prob

Survived                              
Pclass         1         2         3       All
Sex                                           
female  0.968085  0.921053  0.500000  0.742038
male    0.368852  0.157407  0.135447  0.188908
All     0.629630  0.472826  0.242363  0.383838

In [43]:
table_prob = table_prob.drop(('Survived', 'All'), axis=1).drop('All').reset_index()

table_prob

Sex  Survived                    
Pclass                 1         2         3
0       female  0.968085  0.921053  0.500000
1         male  0.368852  0.157407  0.135447

Observe o resultado final do nosso df ao utilizar o método `pd.melt()`:

In [45]:
table_prob.columns

MultiIndex([(     'Sex', ''),
            ('Survived',  1),
            ('Survived',  2),
            ('Survived',  3)],
           names=[None, 'Pclass'])

In [46]:
table_prob

Sex  Survived                    
Pclass                 1         2         3
0       female  0.968085  0.921053  0.500000
1         male  0.368852  0.157407  0.135447

In [63]:
# renomear colunas
table_prob.columns = ['Sex', 1, 2, 3]

In [64]:
table_prob

,Sex,1,2,3
0,female,0.968085,0.921053,0.500000
1,male,0.368852,0.157407,0.135447


In [55]:
table_prob.set_index('Sex').reset_index()

,Sex,1,2,3
0,female,0.968085,0.921053,0.500000
1,male,0.368852,0.157407,0.135447


In [50]:
table_prob.columns

Index(['Sex', 1, 2, 3], dtype='object')

In [66]:
df_melt = pd.melt(table_prob,
       id_vars=['Sex'])
df_melt

,Sex,variable,value
0,female,1,0.968085
1,male,1,0.368852
2,female,2,0.921053
3,male,2,0.157407
4,female,3,0.500000
5,male,3,0.135447


In [67]:
df_melt.columns = ['Sex', 'Pclass_force', 'prob_force']
df_melt

,Sex,Pclass_force,prob_force
0,female,1,0.968085
1,male,1,0.368852
2,female,2,0.921053
3,male,2,0.157407
4,female,3,0.500000
5,male,3,0.135447


Ele converteu as distintas colunas de Pclass em uma coluna com a categoria da classe e outra com seu valor. <br>
Para facilitar o entendimento das novas colunas podemos renomea-las:

In [68]:
pd.melt(table_prob,
       id_vars=['Sex'],
       var_name='Class_melt',
       value_name='porc_of_survived')

,Sex,Class_melt,porc_of_survived
0,female,1,0.968085
1,male,1,0.368852
2,female,2,0.921053
3,male,2,0.157407
4,female,3,0.500000
5,male,3,0.135447


## Transformação de dados

### pd.cut()
O método `pd.cut()` ordena os dados, separa em bins e computa qual grupo cada linha do df pertence. O `pd.cut()` escolherá os bins para serem espaçados uniformemente de acordo com os próprios valores e não com a frequência desses valores.  <br>
Ele é muito utilizado para transformar variáveis contínuas em categóricas. Por exemplo, podemos converter o valor númerico da idade em grupos de criança, jovem, adulto e idoso.
<br><br>
<a href='https://pandas.pydata.org/docs/reference/api/pandas.cut.html'>Parâmetros:</a> <br>
pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)

Ao informar quantidade de grupos o pd.cut() escolhe os bins com o mesmo tamanho de janela :


In [69]:
df['cut_bins'] = pd.cut(df.Age, 4)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins
0,0,3,male,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]"
1,1,1,female,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]"
2,1,3,female,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]"
3,1,1,female,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]"
4,0,3,male,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]"


Podemos passar o nome dos grupos e transformar a variável numérica diretamente em categórica


In [71]:
df['cut_classes'] = pd.cut(df.Age, 4, labels=["jovens", "adultos", "meia-idade", "idosos"])
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
0,0,3,male,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,1,1,female,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
2,1,3,female,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
3,1,1,female,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
4,0,3,male,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos


In [70]:
df.cut_bins.unique()

[(20.315, 40.21], NaN, (40.21, 60.105], (0.34, 20.315], (60.105, 80.0]]
Categories (4, interval[float64]): [(0.34, 20.315] < (20.315, 40.21] < (40.21, 60.105] < (60.105, 80.0]]

In [72]:
df.cut_classes.value_counts()

adultos       385
jovens        179
meia-idade    128
idosos         22
Name: cut_classes, dtype: int64

In [73]:
df.cut_bins.value_counts()

(20.315, 40.21]    385
(0.34, 20.315]     179
(40.21, 60.105]    128
(60.105, 80.0]      22
Name: cut_bins, dtype: int64

Também podemos passar uma lista com os valores de início e fim dos bins:

In [77]:
pd.cut(df.Age, [0,20,60,80]).unique()

[(20.0, 60.0], NaN, (0.0, 20.0], (60.0, 80.0]]
Categories (3, interval[int64]): [(0, 20] < (20, 60] < (60, 80]]

### pd.qcut()
O `pd.qcut()` é utilizado quando queremos discretizar nossos dados em quantis. Ao informar quantidade de grupos o `pd.qcut()` escolhe os bins tal que tenhamos a mesma quantidade de valores em cada grupo.

#### `pd.cut()` x `pd.qcut()`
   * O comando `pd.cut()` cria **caixas equidistantes**, mas a **frequência** das amostras é **desigual** em cada caixa
   * O comando `pd.qcut()` cria **caixas de tamanhos desiguais**, mas a **frequência** das amostras é **igual** em cada caixa.

<br>
Parâmetros:<br>
pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')

In [78]:
pd.cut(df.Age, 4).value_counts()

(20.315, 40.21]    385
(0.34, 20.315]     179
(40.21, 60.105]    128
(60.105, 80.0]      22
Name: Age, dtype: int64

In [79]:
pd.qcut(df.Age, 4).value_counts()

(20.125, 28.0]     183
(0.419, 20.125]    179
(38.0, 80.0]       177
(28.0, 38.0]       175
Name: Age, dtype: int64

In [92]:
pd.qcut(df.Age, 4).value_counts()/714

(20.125, 28.0]     0.256303
(0.419, 20.125]    0.250700
(38.0, 80.0]       0.247899
(28.0, 38.0]       0.245098
Name: Age, dtype: float64

In [94]:
pd.qcut(df.Age, 4).value_counts()/df.Age.notnull().sum()

(20.125, 28.0]     0.256303
(0.419, 20.125]    0.250700
(38.0, 80.0]       0.247899
(28.0, 38.0]       0.245098
Name: Age, dtype: float64

In [91]:
df.Age.shape

(891,)

In [90]:
df.Age.notnull().sum()

714

In [95]:
df.tail(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
871,1,1,female,47.0,1,1,52.5542,D35,S,"(40.21, 60.105]",meia-idade
872,0,1,male,33.0,0,0,5.0000,B51 B53 B55,S,"(20.315, 40.21]",adultos
873,0,3,male,47.0,0,0,9.0000,NaN,S,"(40.21, 60.105]",meia-idade
874,1,2,female,28.0,1,0,24.0000,NaN,C,"(20.315, 40.21]",adultos
875,1,3,female,15.0,0,0,7.2250,NaN,C,"(0.34, 20.315]",jovens
876,0,3,male,20.0,0,0,9.8458,NaN,S,"(0.34, 20.315]",jovens
877,0,3,male,19.0,0,0,7.8958,NaN,S,"(0.34, 20.315]",jovens
878,0,3,male,NaN,0,0,7.8958,NaN,S,NaN,NaN
879,1,1,female,56.0,0,1,83.1583,C50,C,"(40.21, 60.105]",meia-idade
880,1,2,female,25.0,0,1,26.0000,NaN,S,"(20.315, 40.21]",adultos


<a href='https://towardsdatascience.com/discretisation-using-decision-trees-21910483fa4b'>Discretização utilizando decision trees</a>

### pd.get_dummies()

#### variáveis categóricas
Variáveis categóricas são aquelas que representam grupos ou classes dentro dos nossos dados. Elas podem ser de dois tipos:
* ordinais: possuem uma ordem que tem um sentido. Por exemplo, em rendimentos poderíamos ter: classe alta > classe média > classe baixa  
* nominais: não possuem uma ordem válida. Por exemplo: sexo e CEP.

<img src="variaveis_categoricas.jpeg" style="width: 500px">

Dummies são quaisquer variáveis cujos valores são 1 ou 0 para cada observação. O método `pd.get_dummies()` converte as variáveis categóricas em numéricas separando cada categoria em uma coluna única.
<br>
<br>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html">Parâmetros:</a> <br>
pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)

In [97]:
pd.get_dummies(df, columns=['Sex'], drop_first=True)

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes,Sex_male
0,0,3,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos,1
1,1,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos,0
2,1,3,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos,0
3,1,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos,0
4,0,3,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,NaN,S,"(20.315, 40.21]",adultos,1
887,1,1,19.0,0,0,30.0000,B42,S,"(0.34, 20.315]",jovens,0
888,0,3,NaN,1,2,23.4500,NaN,S,NaN,NaN,0
889,1,1,26.0,0,0,30.0000,C148,C,"(20.315, 40.21]",adultos,1


In [125]:
pd.get_dummies(df, columns=['Sex','cut_classes']).head(20)

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,Sex_female,Sex_male,cut_classes_jovens,cut_classes_adultos,cut_classes_meia-idade,cut_classes_idosos
0,0,3,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",0,1,0,1,0,0
1,1,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",1,0,0,1,0,0
3,1,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",1,0,0,1,0,0
4,0,3,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",0,1,0,1,0,0
5,0,3,NaN,0,0,8.4583,NaN,Q,NaN,0,1,0,0,0,0
6,0,1,54.0,0,0,51.8625,E46,S,"(40.21, 60.105]",0,1,0,0,1,0
7,0,3,2.0,3,1,21.0750,NaN,S,"(0.34, 20.315]",0,1,1,0,0,0
8,1,3,27.0,0,2,11.1333,NaN,S,"(20.315, 40.21]",1,0,0,1,0,0
9,1,2,14.0,1,0,30.0708,NaN,C,"(0.34, 20.315]",1,0,1,0,0,0


In [102]:
pd.get_dummies(pd.cut(df.Age, 4))

,"(0.34, 20.315]","(20.315, 40.21]","(40.21, 60.105]","(60.105, 80.0]"
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
886,0,1,0,0
887,1,0,0,0
888,0,0,0,0
889,0,1,0,0


## Multi-index

In [103]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
0,0,3,male,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,1,1,female,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
2,1,3,female,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
3,1,1,female,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
4,0,3,male,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos


Para setar indexes use o método `set_index()` indicando quais as colunas quer utilizar como uma lista.

In [104]:
df_row_index = df.set_index(["Pclass", 'Sex'])
df_row_index

,,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
Pclass,Sex,,,,,,,,,
3,male,0,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,female,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
3,female,1,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
1,female,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
3,male,0,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos
...,...,...,...,...,...,...,...,...,...,...
2,male,0,27.0,0,0,13.0000,NaN,S,"(20.315, 40.21]",adultos
1,female,1,19.0,0,0,30.0000,B42,S,"(0.34, 20.315]",jovens
3,female,0,NaN,1,2,23.4500,NaN,S,NaN,NaN


In [105]:
df_row_index.index

MultiIndex([(3,   'male'),
            (1, 'female'),
            (3, 'female'),
            (1, 'female'),
            (3,   'male'),
            (3,   'male'),
            (1,   'male'),
            (3,   'male'),
            (3, 'female'),
            (2, 'female'),
            ...
            (3,   'male'),
            (3, 'female'),
            (2,   'male'),
            (3,   'male'),
            (3, 'female'),
            (2,   'male'),
            (1, 'female'),
            (3, 'female'),
            (1,   'male'),
            (3,   'male')],
           names=['Pclass', 'Sex'], length=891)

Para acessar elementos:

In [106]:
df_row_index.loc[(3, 'female')]

/home/patricia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Survived   Age  SibSp  Parch     Fare Cabin Embarked  \
Pclass Sex                                                            
3      female         1  26.0      0      0   7.9250   NaN        S   
       female         1  27.0      0      2  11.1333   NaN        S   
       female         1   4.0      1      1  16.7000    G6        S   
       female         0  14.0      0      0   7.8542   NaN        S   
       female         0  31.0      1      0  18.0000   NaN        S   
...                 ...   ...    ...    ...      ...   ...      ...   
       female         0   NaN      8      2  69.5500   NaN        S   
       female         1  15.0      0      0   7.2250   NaN        C   
       female         0  22.0      0      0  10.5167   NaN        S   
       female         0  39.0      0      5  29.1250   NaN        Q   
       female         0   NaN      1      2  23.4500   NaN        S   

                      cut_bins cut_classes  
Pclass Sex                                  
3      female  (20.315, 40.21]     adultos  
       female  (20.315, 40.21]     adultos  
       female   (0.34, 20.315]      jovens  
       female   (0.34, 20.315]      jovens  
       female  (20.315, 40.21]     adultos  
...                        ...         ...  
       female              NaN         NaN  
       female   (0.34, 20.315]      jovens  
       female  (20.315, 40.21]     adultos  
       female  (20.315, 40.21]     adultos  
       female              NaN         NaN  

[144 rows x 9 columns]

In [107]:
df_row_index.reset_index()

,Pclass,Sex,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
0,3,male,0,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,1,female,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
2,3,female,1,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
3,1,female,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
4,3,male,0,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos
...,...,...,...,...,...,...,...,...,...,...,...
886,2,male,0,27.0,0,0,13.0000,NaN,S,"(20.315, 40.21]",adultos
887,1,female,1,19.0,0,0,30.0000,B42,S,"(0.34, 20.315]",jovens
888,3,female,0,NaN,1,2,23.4500,NaN,S,NaN,NaN
889,1,male,1,26.0,0,0,30.0000,C148,C,"(20.315, 40.21]",adultos


In [108]:
df_row_index.reset_index(['Sex'])

,Sex,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
Pclass,,,,,,,,,,
3,male,0,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,female,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
3,female,1,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
1,female,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
3,male,0,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos
...,...,...,...,...,...,...,...,...,...,...
2,male,0,27.0,0,0,13.0000,NaN,S,"(20.315, 40.21]",adultos
1,female,1,19.0,0,0,30.0000,B42,S,"(0.34, 20.315]",jovens
3,female,0,NaN,1,2,23.4500,NaN,S,NaN,NaN


### Multi-index nas colunas

In [109]:
table

Survived              
Pclass        1   2    3  All
Sex                          
female       91  70   72  233
male         45  17   47  109
All         136  87  119  342

Acessando colunas:

In [110]:
table.columns

MultiIndex([('Survived',     1),
            ('Survived',     2),
            ('Survived',     3),
            ('Survived', 'All')],
           names=[None, 'Pclass'])

Como acessar uma coluna:

In [111]:
table[('Survived', 1)]

Sex
female     91
male       45
All       136
Name: (Survived, 1), dtype: int64

Slice usando multi-index

In [113]:
table.index

Index(['female', 'male', 'All'], dtype='object', name='Sex')

In [114]:
table.loc['female', ('Survived', 1):('Survived', 3)]

          Pclass
Survived  1         91
          2         70
          3         72
Name: female, dtype: int64

Para obter o nome das colunas de cada nível hierárquico

In [ ]:
table.columns = []

In [115]:
table

Survived              
Pclass        1   2    3  All
Sex                          
female       91  70   72  233
male         45  17   47  109
All         136  87  119  342

In [116]:
table.columns.get_level_values(0)

Index(['Survived', 'Survived', 'Survived', 'Survived'], dtype='object')

In [117]:
table.columns.get_level_values(1)

Index([1, 2, 3, 'All'], dtype='object', name='Pclass')

In [123]:
nivel_0 = table.columns.get_level_values(0)
nivel_1 = table.columns.get_level_values(1)

[j + '_' + str(nivel_1[i]) for i, j in enumerate(nivel_0)]

['Survived_1', 'Survived_2', 'Survived_3', 'Survived_All']

In [122]:
nivel_1

Index([1, 2, 3, 'All'], dtype='object', name='Pclass')

In [121]:
for i, j in enumerate(nivel_1):
    print('esse eh o indice: ', i)
    print('esse eh o valor: ' , j)

esse eh o indice:  0
esse eh o valor:  1
esse eh o indice:  1
esse eh o valor:  2
esse eh o indice:  2
esse eh o valor:  3
esse eh o indice:  3
esse eh o valor:  All


## Exercícios

1. Baixe os dados de consumo de bebidas por país do <a href="https://www.kaggle.com/justmarkham/alcohol-consumption-by-country">kaggle</a> faça uma análise das informações utilizando os métodos que você já conhece e depois responda:

In [ ]:
drinks = pd.read_csv("data/drinks.csv")
drinks

a. Encontre qual a bebida mais consumida em cada um dos países e a quantidade.

b. Crie um df cujas bebidas estejam agrupadas em uma mesma coluna.

c. Utilizando esse novo df, encontre qual a bebida mais consumida por país e a quantidade.

2. Considere os dados de preço de fechamento e volume das ações que estão dentro de "data/stocks.csv". <br>
a. Escolha um método de python ensinado na aula de hoje para obter um dataframe cujas linhas são os códigos das ações e as colunas são as datas.

b. Com o df original, converta o código das ações para variáveis dummies.

3. Considere os dados do arquivo "german_credit.csv" que contem dados de empréstimos realizados por um banco.<br>
a. Encontre qual a média de empréstimo ("Credit Amount") obtidos considerando o propósito ("Purpose") do empréstimo nas linhas e o sexo ("Sex") nas colunas.

b. Converta as variáveis categóricas em numéricas.

4. Considere o dataset (fake) com testes de aceleração para três carros distintos. Utilize um dos métodos ensinados em aula para criar uma única coluna com os valores das datas e outra com os valores das acelerações.

In [124]:
s = 'Carro A'
x = 'Carro B'
three = 'Carro C'

s_data = [s, 2.5, 2.51, 2.54]
x_data = [x, 2.92, 2.91, 2.93]
three_data = [three, 3.33, 3.31, 3.35]

data = [s_data, x_data, three_data] 
car = pd.DataFrame(data, columns=['car_model', 'Sept 1 9am', 'Sept 1 10am', 'Sept 1 11am'])
car

,car_model,Sept 1 9am,Sept 1 10am,Sept 1 11am
0,Carro A,2.50,2.51,2.54
1,Carro B,2.92,2.91,2.93
2,Carro C,3.33,3.31,3.35


## Referências:
pd.melt(): <br>
https://towardsdatascience.com/shape-tables-like-jelly-with-pandas-melt-and-pivot-f2e13e666d6 <br>
https://pub.towardsai.net/understanding-pandas-melt-pd-melt-362954f8c125